# About:
- this notebooks:
    - optimizes parameters with Optuna with a pruner that observes intermediate results and stop unpromising trials.
    - Approach:
        1. Fix at a high-ish learning rate and at a fixed iteration
        2. perform Hyperparameter Search
        3. Using the best parameters found, reduce learning rate and find the best iteration using early stopping
- insights:
    - **the above approach is effective** - gains are usually seen in step 3

In [1]:
import lightgbm as lgb
import optuna 
from lightgbm import Dataset
from optuna import Trial, visualization
from optuna.samplers import TPESampler

### prepare data

In [2]:
label_colnames = ['h1n1_vaccine', 'seasonal_vaccine']
X = pd.read_csv(r"C:\Users\tanch\Documents\Coding Competitions\DataDriven\Flu Shot Learning\local\data\cleaned_train_set.csv",index_col = "respondent_id")
y = X[label_colnames[1]]
X = X.drop(label_colnames,axis=1)

<IPython.core.display.Javascript object>

In [3]:
# boosting libraries have Dataset class that has memory and speed optimizations
dtrain = Dataset(X, label=y)

# Objective Function
- should take an input Trial instance and return a score

In [4]:
def objective(trial):

    # n_estimators and learning rate are closely tight and will be tuned afterwards
    param = {
        "objective": "binary",
        "learning_rate":0.1,
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "max_depth" : trial.suggest_int("max_depth", 4, 10),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
    
    cv_scores = lgb.cv(param, dtrain, 1000, nfold  = 3, early_stopping_rounds = 20,verbose_eval=False, 
                       callbacks=[pruning_callback])
    
    return cv_scores['auc-mean'][-1]      # last recorded cv score is the best score from early stopping


# create_study

In [5]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="maximize"
)
study.optimize(objective, n_trials=1000)

[I 2021-07-29 19:31:26,678] A new study created in memory with name: no-name-61e871c1-26dd-4068-aca6-d64731bf1d3e
[I 2021-07-29 19:31:27,722] Trial 0 finished with value: 0.8625741011202436 and parameters: {'max_depth': 6, 'colsample_bytree': 0.8448705189311658, 'subsample': 0.6665039870461622, 'min_child_samples': 91, 'lambda_l1': 0.0022255494064781878, 'lambda_l2': 3.987015787570781e-06}. Best is trial 0 with value: 0.8625741011202436.
[I 2021-07-29 19:31:28,705] Trial 1 finished with value: 0.8624438614770368 and parameters: {'max_depth': 8, 'colsample_bytree': 0.7743214679944873, 'subsample': 0.7322971164747668, 'min_child_samples': 93, 'lambda_l1': 0.29129123918541483, 'lambda_l2': 1.3069648221242424e-06}. Best is trial 0 with value: 0.8625741011202436.
[I 2021-07-29 19:31:29,656] Trial 2 finished with value: 0.8619219108269903 and parameters: {'max_depth': 10, 'colsample_bytree': 0.9943979148083687, 'subsample': 0.593235949279995, 'min_child_samples': 77, 'lambda_l1': 4.141081613

[I 2021-07-29 19:31:55,361] Trial 59 finished with value: 0.863295636220638 and parameters: {'max_depth': 9, 'colsample_bytree': 0.5787318392516824, 'subsample': 0.576141312951802, 'min_child_samples': 44, 'lambda_l1': 0.006558765857258209, 'lambda_l2': 0.005703296631966593}. Best is trial 11 with value: 0.8634175138582297.
[I 2021-07-29 19:31:56,227] Trial 60 pruned. Trial was pruned at iteration 91.
[I 2021-07-29 19:31:56,729] Trial 61 pruned. Trial was pruned at iteration 40.
[I 2021-07-29 19:31:57,360] Trial 62 pruned. Trial was pruned at iteration 57.
[I 2021-07-29 19:31:57,889] Trial 63 pruned. Trial was pruned at iteration 41.
[I 2021-07-29 19:31:58,164] Trial 64 pruned. Trial was pruned at iteration 16.
[I 2021-07-29 19:31:58,679] Trial 65 pruned. Trial was pruned at iteration 40.
[I 2021-07-29 19:31:58,933] Trial 66 pruned. Trial was pruned at iteration 13.
[I 2021-07-29 19:31:59,158] Trial 67 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:31:59,388] Trial 68 prune

[I 2021-07-29 19:32:27,308] Trial 143 pruned. Trial was pruned at iteration 23.
[I 2021-07-29 19:32:27,552] Trial 144 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:32:28,028] Trial 145 pruned. Trial was pruned at iteration 36.
[I 2021-07-29 19:32:28,583] Trial 146 pruned. Trial was pruned at iteration 40.
[I 2021-07-29 19:32:28,825] Trial 147 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:32:29,112] Trial 148 pruned. Trial was pruned at iteration 12.
[I 2021-07-29 19:32:29,489] Trial 149 pruned. Trial was pruned at iteration 20.
[I 2021-07-29 19:32:29,739] Trial 150 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:32:30,294] Trial 151 pruned. Trial was pruned at iteration 40.
[I 2021-07-29 19:32:30,548] Trial 152 pruned. Trial was pruned at iteration 12.
[I 2021-07-29 19:32:31,059] Trial 153 pruned. Trial was pruned at iteration 40.
[I 2021-07-29 19:32:31,374] Trial 154 pruned. Trial was pruned at iteration 13.
[I 2021-07-29 19:32:31,882] Trial 155 pr

[I 2021-07-29 19:33:00,828] Trial 223 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:33:01,985] Trial 224 finished with value: 0.8632381046599916 and parameters: {'max_depth': 10, 'colsample_bytree': 0.5439958219028861, 'subsample': 0.9448290421059372, 'min_child_samples': 41, 'lambda_l1': 0.017943148817174612, 'lambda_l2': 0.007468743581016311}. Best is trial 162 with value: 0.8634888365505201.
[I 2021-07-29 19:33:02,321] Trial 225 pruned. Trial was pruned at iteration 20.
[I 2021-07-29 19:33:02,720] Trial 226 pruned. Trial was pruned at iteration 27.
[I 2021-07-29 19:33:02,977] Trial 227 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:33:03,365] Trial 228 pruned. Trial was pruned at iteration 20.
[I 2021-07-29 19:33:04,056] Trial 229 pruned. Trial was pruned at iteration 57.
[I 2021-07-29 19:33:04,476] Trial 230 pruned. Trial was pruned at iteration 26.
[I 2021-07-29 19:33:05,212] Trial 231 pruned. Trial was pruned at iteration 66.
[I 2021-07-29 19:33:05,547] T

[I 2021-07-29 19:33:33,025] Trial 309 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:33:33,402] Trial 310 pruned. Trial was pruned at iteration 23.
[I 2021-07-29 19:33:33,855] Trial 311 pruned. Trial was pruned at iteration 26.
[I 2021-07-29 19:33:34,151] Trial 312 pruned. Trial was pruned at iteration 15.
[I 2021-07-29 19:33:34,431] Trial 313 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:33:34,786] Trial 314 pruned. Trial was pruned at iteration 16.
[I 2021-07-29 19:33:35,008] Trial 315 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:33:35,299] Trial 316 pruned. Trial was pruned at iteration 15.
[I 2021-07-29 19:33:35,675] Trial 317 pruned. Trial was pruned at iteration 19.
[I 2021-07-29 19:33:36,039] Trial 318 pruned. Trial was pruned at iteration 23.
[I 2021-07-29 19:33:36,291] Trial 319 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:33:36,540] Trial 320 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:33:37,045] Trial 321 pr

...

[I 2021-07-29 19:35:05,417] Trial 567 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:05,798] Trial 568 pruned. Trial was pruned at iteration 20.
[I 2021-07-29 19:35:06,167] Trial 569 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:06,474] Trial 570 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:06,933] Trial 571 pruned. Trial was pruned at iteration 30.
[I 2021-07-29 19:35:07,378] Trial 572 pruned. Trial was pruned at iteration 22.
[I 2021-07-29 19:35:07,658] Trial 573 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:07,945] Trial 574 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:08,213] Trial 575 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:08,573] Trial 576 pruned. Trial was pruned at iteration 17.
[I 2021-07-29 19:35:08,877] Trial 577 pruned. Trial was pruned at iteration 13.
[I 2021-07-29 19:35:09,141] Trial 578 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:09,461] Trial 579 pr

[I 2021-07-29 19:35:35,176] Trial 653 pruned. Trial was pruned at iteration 19.
[I 2021-07-29 19:35:35,494] Trial 654 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:35,784] Trial 655 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:36,112] Trial 656 pruned. Trial was pruned at iteration 15.
[I 2021-07-29 19:35:36,388] Trial 657 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:36,715] Trial 658 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:37,004] Trial 659 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:37,280] Trial 660 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:37,696] Trial 661 pruned. Trial was pruned at iteration 19.
[I 2021-07-29 19:35:38,023] Trial 662 pruned. Trial was pruned at iteration 15.
[I 2021-07-29 19:35:38,411] Trial 663 pruned. Trial was pruned at iteration 22.
[I 2021-07-29 19:35:38,751] Trial 664 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:35:39,065] Trial 665 pr

[I 2021-07-29 19:36:05,189] Trial 742 pruned. Trial was pruned at iteration 13.
[I 2021-07-29 19:36:05,526] Trial 743 pruned. Trial was pruned at iteration 15.
[I 2021-07-29 19:36:05,851] Trial 744 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:06,209] Trial 745 pruned. Trial was pruned at iteration 17.
[I 2021-07-29 19:36:06,476] Trial 746 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:06,809] Trial 747 pruned. Trial was pruned at iteration 14.
[I 2021-07-29 19:36:07,130] Trial 748 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:07,514] Trial 749 pruned. Trial was pruned at iteration 19.
[I 2021-07-29 19:36:07,817] Trial 750 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:08,152] Trial 751 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:08,543] Trial 752 pruned. Trial was pruned at iteration 20.
[I 2021-07-29 19:36:08,819] Trial 753 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:09,162] Trial 754 pr

[I 2021-07-29 19:36:36,600] Trial 831 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:37,009] Trial 832 pruned. Trial was pruned at iteration 23.
[I 2021-07-29 19:36:37,329] Trial 833 pruned. Trial was pruned at iteration 13.
[I 2021-07-29 19:36:37,670] Trial 834 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:37,954] Trial 835 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:38,267] Trial 836 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:38,564] Trial 837 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:39,051] Trial 838 pruned. Trial was pruned at iteration 19.
[I 2021-07-29 19:36:39,474] Trial 839 pruned. Trial was pruned at iteration 23.
[I 2021-07-29 19:36:39,907] Trial 840 pruned. Trial was pruned at iteration 20.
[I 2021-07-29 19:36:40,172] Trial 841 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:40,461] Trial 842 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:36:40,801] Trial 843 pr

[I 2021-07-29 19:37:07,086] Trial 917 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:37:07,457] Trial 918 pruned. Trial was pruned at iteration 18.
[I 2021-07-29 19:37:07,901] Trial 919 pruned. Trial was pruned at iteration 20.
[I 2021-07-29 19:37:08,202] Trial 920 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:37:08,505] Trial 921 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:37:08,895] Trial 922 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:37:09,215] Trial 923 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:37:09,575] Trial 924 pruned. Trial was pruned at iteration 15.
[I 2021-07-29 19:37:09,927] Trial 925 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:37:10,300] Trial 926 pruned. Trial was pruned at iteration 17.
[I 2021-07-29 19:37:10,588] Trial 927 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:37:10,899] Trial 928 pruned. Trial was pruned at iteration 10.
[I 2021-07-29 19:37:11,530] Trial 929 pr

In [6]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 1000
Best trial:
  Value: 0.8634888365505201
  Params: 
    max_depth: 8
    colsample_bytree: 0.5408815427763269
    subsample: 0.8466638436024809
    min_child_samples: 37
    lambda_l1: 0.0001634213220638466
    lambda_l2: 0.0019856133446647076


## Optimize n_estimators and learning rate
- set to high iterations and low learning rate

In [10]:
param = trial.params
param.update({"learning_rate":0.01,
              "objective": "binary",
              "metric": "auc"})
param

{'max_depth': 8,
 'colsample_bytree': 0.5408815427763269,
 'subsample': 0.8466638436024809,
 'min_child_samples': 37,
 'lambda_l1': 0.0001634213220638466,
 'lambda_l2': 0.0019856133446647076,
 'learning_rate': 0.01,
 'objective': 'binary',
 'metric': 'auc'}

In [11]:
cv_scores = lgb.cv(param, dtrain, 10000, nfold  = 5, early_stopping_rounds = 50,verbose_eval=False)['auc-mean']

In [12]:
print(f"Best Iteration {len(cv_scores)} at {cv_scores[-1]}")

Best Iteration 896 at 0.8644408064138457


# Train and export

In [16]:
best_param = {
    'max_depth': 8,
    'colsample_bytree': 0.5408815427763269,
    'subsample': 0.8466638436024809,
    'min_child_samples': 37,
    'lambda_l1': 0.0001634213220638466,
    'lambda_l2': 0.0019856133446647076,
    'learning_rate': 0.01,
    'objective': 'binary',
    'metric': 'auc',
    "num_iterations":896
}

In [17]:
model = lgb.LGBMClassifier()
model.set_params(**best_param)
model.fit(X,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.5408815427763269, importance_type='split',
               lambda_l1=0.0001634213220638466, lambda_l2=0.0019856133446647076,
               learning_rate=0.01, max_depth=8, metric='auc',
               min_child_samples=37, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_iterations=896, num_leaves=31,
               objective='binary', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=0.8466638436024809,
               subsample_for_bin=200000, subsample_freq=0)

In [18]:
import joblib
joblib.dump(model, r"C:\Users\tanch\Documents\Coding Competitions\DataDriven\Flu Shot Learning\local\model\lightgbm seasonsal_vaccine 0.pkl")
# gbm_pickle = joblib.load(r"C:\Users\tanch\Documents\Coding Competitions\DataDriven\Flu Shot Learning\local\model\lightgbm 0.pkl")

['C:\\Users\\tanch\\Documents\\Coding Competitions\\DataDriven\\Flu Shot Learning\\local\\model\\lightgbm seasonsal_vaccine 0.pkl']